### Dataset Reviews

> Neste notebook ocorrerá as análises e alterações necessárias para tornar o dataframe Reviews confiável para modelagem dimensional tais como comentários acerca de determinadas decisões.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

PATH_RAW = '../input/raw/'
PATH_TRUSTED = '../output/pd/trusted/'

ds_reviews = pd.read_csv(PATH_RAW + 'olist_order_reviews_dataset.csv')
ds_orders = pd.read_csv(PATH_TRUSTED + 'orders_trusted.csv')

#### Análise do dataset Order Reviews

In [2]:
ds_reviews.head(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13


**Análise das Variáveis/Colunas**

- **review_score**: Variável Qualitativa Ordinal
- **review_comment_title**: Variável Qualitativa Nominal
- **review_comment_message**: Variável Qualitativa Nominal
- **review_creation_date**: Variável Qualitativa Ordinal
- **review_answer_timestamp**: Variável Qualitativa Ordinal

#### Análise Valores Nulos/Inconsistências

In [3]:
ds_reviews.review_score.value_counts()

review_score
5    57328
4    19142
1    11424
3     8179
2     3151
Name: count, dtype: int64

In [4]:
ds_reviews.isnull().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

> Aqui achei interessante invés de deixar um título e mensagem Nulos, **definir um padrão para ser exibido**.

In [5]:
ds_reviews.fillna({'review_comment_title':'Sem Titulo', 'review_comment_message':'Sem Comentários'}, inplace=True)

In [6]:
ds_reviews.isnull().sum()

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64

#

#### Verificação de Duplicatas

> Agora com os dados devidamente tratados, começaremos a fazer as nossas modificações.

> Porém antes removerei os dados duplicados referentes a `order_id` pois de acordo com minha análise feita, faz sentido um mesmo cliente fazer reviews diferentes `(review_id duplicados)` porém sempre será de pedidos diferentes, pois sempre é gerado um novo `order_id` a compras distintas.

In [7]:
ds_reviews.shape

(99224, 7)

In [8]:
ds_reviews.duplicated(subset=['order_id'], keep='first').value_counts()

False    98673
True       551
Name: count, dtype: int64

In [9]:
ds_reviews.drop_duplicates(subset=['order_id'], inplace=True)

In [10]:
ds_reviews.duplicated(subset=['order_id'], keep='first').value_counts()

False    98673
Name: count, dtype: int64

> Agora, criarei um novo `review_id`, pois o id deve ser único, também percebi que conseguimos extrair o cliente diretamente dataset `orders`, ou seja, não importa se os valores deste dataset estão duplicados, conseguimos analisar da mesma forma através do dataset `orders`

In [11]:
ds_reviews.drop(columns=['review_id'], inplace=True)
ds_reviews['review_id'] = range(1, len(ds_reviews) + 1)

In [12]:
ds_reviews

,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_id
0,73fc7af87114b39712e6da79b0a377eb,4,Sem Titulo,Sem Comentários,2018-01-18 00:00:00,2018-01-18 21:46:59,1
1,a548910a1c6147796b98fdf73dbeba33,5,Sem Titulo,Sem Comentários,2018-03-10 00:00:00,2018-03-11 03:05:13,2
2,f9e4b658b201a9f2ecdecbb34bed034b,5,Sem Titulo,Sem Comentários,2018-02-17 00:00:00,2018-02-18 14:36:24,3
3,658677c97b385a9be170737859d3511b,5,Sem Titulo,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,4
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,Sem Titulo,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5
...,...,...,...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,5,Sem Titulo,Sem Comentários,2018-07-07 00:00:00,2018-07-14 17:18:30,98669
99220,22ec9f0669f784db00fa86d035cf8602,5,Sem Titulo,Sem Comentários,2017-12-09 00:00:00,2017-12-11 20:06:42,98670
99221,55d4004744368f5571d1f590031933e4,5,Sem Titulo,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43,98671
99222,7725825d039fc1f0ceb7635e3f7d9206,4,Sem Titulo,Sem Comentários,2018-07-01 00:00:00,2018-07-02 12:59:13,98672


#

#### Padronização dos Indexes

> Com o nosso novo `review_id` podemos substituir o `order_id` do nosso dataset `review` com o `order_id_int` criado no notebook de [Payment](./payments_to_trusted.ipynb).

In [13]:
order_id_map = ds_orders.set_index('order_id')['order_id_int']
order_id_map

order_id
e481f51cbdc54678b7cc49136f2d6af7        1
53cdb2fc8bc7dce0b6741e2150273451        2
47770eb9100c2d0c44946d9cf07ec65d        3
949d5b44dbf5de918fe9c16f97b45f8a        4
ad21c59c0840e6cb83a9ceb5573f8159        5
                                    ...  
9c5dedf39a927c1b2549525ed64a053c    99437
63943bddc261676b46f01ca7ac2f7bd8    99438
83c1379a015df1e13d02aae0204711ab    99439
11c177c8e97725db2631073c19f07b62    99440
66dea50a8b16d9b4dee7af250b4be1a5    99441
Name: order_id_int, Length: 99441, dtype: int64

In [14]:
ds_reviews['order_id'] = ds_reviews['order_id'].map(order_id_map)
ds_reviews

,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_id
0,1872,4,Sem Titulo,Sem Comentários,2018-01-18 00:00:00,2018-01-18 21:46:59,1
1,90489,5,Sem Titulo,Sem Comentários,2018-03-10 00:00:00,2018-03-11 03:05:13,2
2,78883,5,Sem Titulo,Sem Comentários,2018-02-17 00:00:00,2018-02-18 14:36:24,3
3,27558,5,Sem Titulo,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,4
4,79506,5,Sem Titulo,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5
...,...,...,...,...,...,...,...
99219,66507,5,Sem Titulo,Sem Comentários,2018-07-07 00:00:00,2018-07-14 17:18:30,98669
99220,84919,5,Sem Titulo,Sem Comentários,2017-12-09 00:00:00,2017-12-11 20:06:42,98670
99221,9818,5,Sem Titulo,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43,98671
99222,29007,4,Sem Titulo,Sem Comentários,2018-07-01 00:00:00,2018-07-02 12:59:13,98672


#

#### Carregamento do dataset Review para a trusted

> Agora com nosso dataset review atualizado com seu novo índice `review_id` e devidas alterações visando a consistência, podemos carregar os datasets alterados para a nossa camada `trusted`

In [15]:
ds_reviews = ds_reviews.convert_dtypes()
ds_orders = ds_orders.convert_dtypes()

ds_reviews.to_csv(PATH_TRUSTED + 'reviews_trusted.csv', index=False)
ds_orders.to_csv(PATH_TRUSTED + 'orders_trusted.csv', index=False)